
## DESCRIPTION

This notebook fits a series of generalized linear mixed models

***
### SETUP

Load the setup script

In [3]:
from aqi.utils.common_setup import *

Load the functions to :

- Fit the models
- Save the models
- Load the models

In [4]:
from aqi.utils.functions import fit_models, save_fitted_models, load_fitted_models

Import the cleaned dataset

In [5]:
data_folder = os.path.join(os.getcwd(), '..', 'data')
data = pd.read_csv(os.path.join(data_folder, 'clean_data.csv'))

In [6]:
# Count the number of unique hive_id values
print(f"Number of unique hives: {data['hid'].nunique()}")
print(f"Number of unique operations: {data['operation_id'].nunique()}")

Number of unique hives: 103477
Number of unique operations: 17


In [5]:
len(data)

112626

In [5]:
data.columns

Index(['hid', 'creation_date', 'operation_id', 'death_date', 'season',
       'season_start_month', 'season_start_day', 'death_next_season',
       'hive_age_next_season', 'aqhi_average', 'prcp_average', 'wspd_average',
       'ndvi_average', 'tavg_average', 'o3_average', 'aqhi_skew', 'prcp_skew',
       'wspd_skew', 'ndvi_skew', 'tavg_skew', 'o3_skew', 'aqhi_max',
       'prcp_max', 'wspd_max', 'ndvi_max', 'tavg_max', 'o3_max', 'region',
       'wspd_average_og', 'tavg_average_og', 'prcp_average_og',
       'aqhi_average_og'],
      dtype='object')

### IF PackageNotInstalledError: The R package "lme4" is not installed. 


See README for instruction


```
from rpy2.robjects.packages import importr


utils = importr('utils')
utils.chooseCRANmirror(ind=12)
utils.install_packages('lme4')
```

### FIT GLMM

### Define model formulas

There are 33 models starting from model 0

For features, I use the log of precipitation --> prcp_average

The rest is original data --> anything og

See 02_CleanData.ipynb for details

In [36]:
model_formulas = [
    
    # Null model ------------------------------------------------

    # model0
    "death_next_season ~ 1 + (1|region)",

    # Models with aqhi average and region -----------------------

    # model1
    "death_next_season ~ aqhi_average_og + ndvi_average + (aqhi_average|region)",

    # model2
    "death_next_season ~ aqhi_average_og * ndvi_average + (aqhi_average_og|region)",

    # model3
    "death_next_season ~ aqhi_average_og + ndvi_average + tavg_average_og + prcp_average + wspd_average_og + (aqhi_average_og|region)",

    # model4
    "death_next_season ~ aqhi_average_og * wspd_average_og + ndvi_average + prcp_average + tavg_average_og + (aqhi_average_og|region)",

    # model5
    "death_next_season ~ aqhi_average_og * ndvi_average + wspd_average_og + (ndvi_average|region)",

    # model6
    "death_next_season ~ aqhi_average_og * wspd_average_og * tavg_average_og + (aqhi_average_og|region)",

    # model7
    "death_next_season ~ aqhi_average_og * wspd_average_og * ndvi_average + (ndvi_average|region)",

    # Models with aqhi average and operation id -----------------

    # model8
    "death_next_season ~ aqhi_average_og * ndvi_average + (1|operation_id)",

    # model9
    "death_next_season ~ aqhi_average_og * ndvi_average + (aqhi_average_og|operation_id)",

    # model10
    "death_next_season ~ aqhi_average_og + ndvi_average + wspd_average_og + (1|operation_id)",

    # model11
    "death_next_season ~ aqhi_average_og + ndvi_average + wspd_average_og + (aqhi_average_og|operation_id)",

    # model12
    "death_next_season ~ aqhi_average_og * wspd_average_og + ndvi_average + (aqhi_average_og|operation_id)",

    # model13
    "death_next_season ~ aqhi_average_og * ndvi_average + wspd_average_og + (1|operation_id)",

    # model14
    "death_next_season ~ aqhi_average_og * ndvi_average * wspd_average_og + (1|operation_id)",

    # model15
    "death_next_season ~ aqhi_average_og * ndvi_average * wspd_average_og + (aqhi_average_og|operation_id)",

    # model16
    "death_next_season ~ aqhi_average_og + ndvi_average + wspd_average_og + tavg_average_og + prcp_average + (aqhi_average_og|operation_id)",

    # model17
    "death_next_season ~ aqhi_average_og * ndvi_average + tavg_average_og * prcp_average + wspd_average_og + (aqhi_average_og|operation_id)",

    # Models with aqhi skew and region --------------------------
    
    # model18
    "death_next_season ~ aqhi_skew + ndvi_skew + (aqhi_skew|region)",

    # model19
    "death_next_season ~ aqhi_skew * ndvi_skew + (aqhi_skew|region)",

    # model20
    "death_next_season ~ aqhi_skew * ndvi_average + wspd_average_og + (aqhi_skew|region)",

    # model21
    "death_next_season ~ aqhi_skew * ndvi_average * wspd_average_og + (aqhi_skew|region)",

    # Models with aqhi max ---------------------------

    # model22
    "death_next_season ~ aqhi_max * ndvi_average + wspd_average_og + (ndvi_average|region)",

    # model23
    "death_next_season ~ aqhi_max * ndvi_average * wspd_average_og + (ndvi_average|region)",

    # model24
    "death_next_season ~ aqhi_max * ndvi_average + wspd_average_og + (aqhi_average_og|operation_id)",

    # model25
    "death_next_season ~ aqhi_max * ndvi_average * wspd_average_og + (aqhi_average_og|operation_id)",

    # Models with ndvi skew --------------------------

    # model26
    "death_next_season ~ aqhi_average_og * ndvi_skew + wspd_average_og + (aqhi_average_og|region)",

    # model27
    "death_next_season ~ aqhi_average_og * ndvi_skew * wspd_average_og + (aqhi_average_og|region)",

    # model28
    "death_next_season ~ aqhi_average_og * ndvi_skew * wspd_average_og + (aqhi_average_og|operation_id)",

    # Models with O3 and operation id ---------------------------

    # model29
    "death_next_season ~ ndvi_average + o3_average + (o3_average|operation_id)",

    # model30
    "death_next_season ~ ndvi_average * o3_average + (o3_average|operation_id)",

    # model31
    "death_next_season ~ ndvi_average * o3_average + wspd_average_og + (o3_average|operation_id)",

    # model32
    "death_next_season ~ ndvi_average * o3_average * wspd_average_og + (o3_average|operation_id)"

]

### Standardize all variables

In [33]:
from sklearn.preprocessing import StandardScaler

# Variables
variables_to_scale = [
    'aqhi_average_og', 'ndvi_average', 'wspd_average_og', 
    'tavg_average_og', 'prcp_average', 'o3_average',
    'aqhi_skew', 'aqhi_max', 'ndvi_skew'
]

# Initialize the scaler
scaler = StandardScaler()

# Overwrite original variables names with scaled variables
data[variables_to_scale] = scaler.fit_transform(data[variables_to_scale])

In [34]:
print(data[variables_to_scale].describe())

       aqhi_average_og  ndvi_average  wspd_average_og  tavg_average_og  \
count     1.126260e+05  1.126260e+05     1.126260e+05     1.126260e+05   
mean     -1.615071e-16 -1.736201e-16     6.056515e-17     3.633909e-17   
std       1.000004e+00  1.000004e+00     1.000004e+00     1.000004e+00   
min      -2.656698e+00 -8.433916e+00    -5.001226e+00    -5.900251e+00   
25%      -7.633126e-01 -4.662709e-01    -5.466750e-01    -7.052172e-01   
50%      -3.487361e-01 -1.513841e-01     1.065399e-02    -1.949752e-01   
75%       9.614525e-01  3.347263e-01     5.835905e-01     6.022519e-01   
max       2.967133e+00  1.472707e+01     8.448379e+00     3.778067e+00   

       prcp_average    o3_average     aqhi_skew      aqhi_max     ndvi_skew  
count  1.126260e+05  1.126260e+05  1.121070e+05  1.126260e+05  1.121570e+05  
mean   4.037676e-17  4.602951e-16 -5.678916e-17  1.049796e-16  3.745400e-16  
std    1.000004e+00  1.000004e+00  1.000004e+00  1.000004e+00  1.000004e+00  
min   -5.548384e+00 -

Inspect how many models

In [39]:
len(model_formulas)

33

In [40]:
data.columns

Index(['hid', 'creation_date', 'operation_id', 'death_date', 'season',
       'season_start_month', 'season_start_day', 'death_next_season',
       'hive_age_next_season', 'aqhi_average', 'prcp_average', 'wspd_average',
       'ndvi_average', 'tavg_average', 'o3_average', 'aqhi_skew', 'prcp_skew',
       'wspd_skew', 'ndvi_skew', 'tavg_skew', 'o3_skew', 'aqhi_max',
       'prcp_max', 'wspd_max', 'ndvi_max', 'tavg_max', 'o3_max', 'region',
       'wspd_average_og', 'tavg_average_og', 'prcp_average_og',
       'aqhi_average_og'],
      dtype='object')

### Fit the models and save (**run once**)

Fitting all models takes **approximately 1.2 hours.**

Run once and save in ```aqi/outputs``` folder so you can reimport in session.

File is called ```fitted_GLMMs.pkl```

In [41]:
fitted_models, model_summaries = fit_models(
    data=data,
    formulas=model_formulas
)

save_fitted_models(
    fitted_models=fitted_models,
    filename="fitted_GLMMs.pkl"
)

save_fitted_models(
    fitted_models=model_summaries,
    filename="summaries_GLMMs.pkl"
)

Fitting Models:   0%|          | 0/33 [00:00<?, ?it/s]

Fitting model0 with formula: death_next_season ~ 1 + (1|region)


Fitting Models:   3%|▎         | 1/33 [00:17<09:33, 17.93s/it]

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: death_next_season~1+(1|region)

Family: binomial	 Inference: parametric

Number of observations: 112626	 Groups: {'region': 9.0}

Log-likelihood: -71157.617 	 AIC: 142319.235

Random effects:

               Name    Var    Std
region  (Intercept)  0.979  0.989

No random effect correlations specified

Fixed effects:

Fitting model1 with formula: death_next_season ~ aqhi_average_og + ndvi_average + (aqhi_average|region)


Fitting Models:   6%|▌         | 2/33 [01:12<20:26, 39.56s/it]

...

Fitting Models:  97%|█████████▋| 32/33 [35:39<01:06, 66.58s/it]

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: death_next_season~ndvi_average*o3_average+wspd_average_og+(o3_average|operation_id)

Family: binomial	 Inference: parametric

Number of observations: 112626	 Groups: {'operation_id': 17.0}

Log-likelihood: -64589.811 	 AIC: 129195.623

Random effects:

                     Name    Var    Std
operation_id  (Intercept)  3.482  1.866
operation_id   o3_average  3.883  1.970

                      IV1         IV2   Corr
operation_id  (Intercept)  o3_average  0.685

Fixed effects:

Fitting model32 with formula: death_next_season ~ ndvi_average * o3_average * wspd_average_og + (o3_average|operation_id)


Fitting Models: 100%|██████████| 33/33 [37:32<00:00, 68.27s/it]

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: death_next_season~ndvi_average*o3_average*wspd_average_og+(o3_average|operation_id)

Family: binomial	 Inference: parametric

Number of observations: 112626	 Groups: {'operation_id': 17.0}

Log-likelihood: -64516.287 	 AIC: 129054.575

Random effects:

                     Name    Var    Std
operation_id  (Intercept)  3.464  1.861
operation_id   o3_average  4.042  2.011

                      IV1         IV2  Corr
operation_id  (Intercept)  o3_average  0.69

Fixed effects:



Models saved to /Users/maximeff-nectar/Documents/ledge-nectar/aqi/outputs/fitted_GLMMs.pkl.
Models saved to /Users/maximeff-nectar/Documents/ledge-nectar/aqi/outputs/summaries_GLMMs.pkl.


### Load models in session

In [10]:
fitted_models = load_fitted_models("fitted_GLMMs.pkl")

Models loaded from /Users/maximeff-nectar/Documents/ledge-nectar/aqi/outputs/fitted_GLMMs.pkl.


In [42]:
# Initialize variables
best_aic = float('inf')
best_model = None
best_model_name = None

# Loop over the fitted models dict
for model_name, model in fitted_models.items():
    
    # Check if the model has warnings
    if len(model.warnings) == 0:
        # Get the AIC for the current model
        current_aic = model.AIC
        
        # Compare with the best AIC found so far
        if current_aic < best_aic:
            print(f"Current best model is {model_name} with an AIC of {current_aic}")
            best_aic = current_aic
            best_model = model
            best_model_name = model_name
    else:
        print(f"Model {model_name} did not converge due to warnings: {model.warnings}")

Current best model is model0 with an AIC of 142319.23490165427
Current best model is model1 with an AIC of 138946.95945308142
Current best model is model2 with an AIC of 138683.27267033744
Current best model is model3 with an AIC of 136834.70097385795
Current best model is model4 with an AIC of 136833.73152819887
Current best model is model8 with an AIC of 132924.32125198081
Current best model is model9 with an AIC of 131344.03276013533
Current best model is model15 with an AIC of 131268.5978089567
Current best model is model16 with an AIC of 129516.05600307278
Current best model is model17 with an AIC of 129511.72722862924
Current best model is model29 with an AIC of 129405.29649029924
Current best model is model30 with an AIC of 129319.57520043233
Current best model is model31 with an AIC of 129195.6229296742
Current best model is model32 with an AIC of 129054.57482102417


In [43]:
print(best_model.fit())

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: death_next_season~ndvi_average*o3_average*wspd_average_og+(o3_average|operation_id)

Family: binomial	 Inference: parametric

Number of observations: 112626	 Groups: {'operation_id': 17.0}

Log-likelihood: -64516.287 	 AIC: 129054.575

Random effects:

                     Name    Var    Std
operation_id  (Intercept)  3.464  1.861
operation_id   o3_average  4.042  2.011

                      IV1         IV2  Corr
operation_id  (Intercept)  o3_average  0.69

Fixed effects:

                                         Estimate  2.5_ci  97.5_ci     SE  \
(Intercept)                                -0.149  -1.040    0.743  0.455   
ndvi_average                               -0.413  -0.432   -0.393  0.010   
o3_average                                  1.243   0.287    2.200  0.488   
wspd_average_og                            -0.126  -0.142   -0.109  0.009   
ndvi_average:o3_average                    -0.106  -0.126   -0.085  0

Save the best model output

In [44]:
project_root = os.path.dirname(os.getcwd())
output_dir = os.path.join(project_root, 'outputs')    
filepath = os.path.join(output_dir, "GLMM_best_model.joblib")

save_model(best_model, filepath = filepath)